In [115]:
import json
import numpy as np
import os
import pandas as pd
import urllib3
import urllib.request
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import tensorflow as tf
#from tensorflow.keras.backend import set_session
#from keras.backend.tensorflow_backend import *

In [116]:
#coins = ['BTC', 'LTC', 'ETH', 'XRP']
coins = ['XRP']

In [117]:
# ratio of train to test split (originally at 0.8)
train_ratio = 0.9

In [118]:
from datetime import datetime, timedelta
from time import mktime

delta_days = 90

t = datetime.now()
end_tm = np.int(mktime(t.timetuple()))
tpast = t - timedelta(days=delta_days)
start_tm = np.int(mktime(tpast.timetuple()))
#start_tm = mktime((t - timedelta(days=delta_days)).tuple())
period_step = 300

In [119]:
df_list=[]

container = []

for coin in coins:
    url = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_'+coin+'&start={0}&end={1}&period={2}'.format(start_tm, end_tm, period_step)
    openUrl = urllib.request.urlopen(url)
    r = openUrl.read()
    openUrl.close()
    d = json.loads(r.decode())
    dfx = pd.DataFrame(d)
    dfx['coin'] = coin
    container.append(dfx)
    print(pd.DataFrame(d).shape)
    
container = pd.concat(container, axis = 0)
print('Returned length of {0} coins: {1}'.format(len(coins), container.shape))


# format
original_columns=[u'close', u'date', u'high', u'low', u'open']
new_columns = ['Close','Timestamp','High','Low','Open']
container = container.loc[:,original_columns]
container.columns = new_columns
container.to_csv('data/bitcoin2015to2017.csv',index=None)

# export out
h5path = 'data/4coins_17apr2021.h5'
container.to_hdf(h5path, key='df', mode='w')
#container.to_csv('data/4coins_17apr2021.csv',index=None)

(25920, 8)
Returned length of 1 coins: (25920, 9)


In [120]:

import numpy as np
import pandas as pd
 
class PastSampler:
    '''
    Forms training samples for predicting future values from past value
    '''
     
    def __init__(self, N, K, sliding_window = True):
        '''
        Predict K future sample using N previous samples
        '''
        self.K = K
        self.N = N
        self.sliding_window = sliding_window
 
    def transform(self, A):
        M = self.N + self.K     #Number of samples per row (sample + target)
        #indexes
        if self.sliding_window:
            I = np.arange(M) + np.arange(A.shape[0] - M + 1).reshape(-1, 1)
        else:
            if A.shape[0]%M == 0:
                I = np.arange(M)+np.arange(0,A.shape[0],M).reshape(-1,1)
                
            else:
                I = np.arange(M)+np.arange(0,A.shape[0] -M,M).reshape(-1,1)
            
        B = A[I].reshape(-1, M * A.shape[1], A.shape[2])
        ci = self.N * A.shape[1]    #Number of features per sample
        return B[:, :ci], B[:, ci:] #Sample matrix, Target matrix

#data file path
dfp = 'data/bitcoin2015to2017.csv'

#Columns of price data to use
columns = ['Close']
df = pd.read_csv(dfp)
time_stamps = df['Timestamp']
df = df.loc[:,columns]
original_df = pd.read_csv(dfp).loc[:,columns]

In [121]:
file_name='bitcoin2015to2017_close.h5'

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# normalization
for c in columns:
    df[c] = scaler.fit_transform(df[c].values.reshape(-1,1))
    
#Features are input sample dimensions(channels)
A = np.array(df)[:,None,:]
original_A = np.array(original_df)[:,None,:]
time_stamps = np.array(time_stamps)[:,None,None]

#Make samples of temporal sequences of pricing data (channel)
NPS, NFS = 256, 16        #Number of past and future samples  -original at 16
ps = PastSampler(NPS, NFS, sliding_window=True)
B, Y = ps.transform(A)
input_times, output_times = ps.transform(time_stamps)
original_B, original_Y = ps.transform(original_A)

import h5py
with h5py.File(file_name, 'w') as f:
    f.create_dataset("inputs", data = B)
    f.create_dataset('outputs', data = Y)
    f.create_dataset("input_times", data = input_times)
    f.create_dataset('output_times', data = output_times)
    f.create_dataset("original_datas", data=np.array(original_df))
    f.create_dataset('original_inputs',data=original_B)
    f.create_dataset('original_outputs',data=original_Y)

In [122]:
time_stamps[25920]

IndexError: index 25920 is out of bounds for axis 0 with size 25920

In [ ]:
len(input_times[0])

In [ ]:
len(output_times[0])

In [ ]:
(1618720500 - 1618691400)/60/60

In [ ]:
input_times[94][255]

In [ ]:
(1618696200 - 1618691400)/60

In [ ]:
output_times[94][15]

In [ ]:
len(time_stamps)

In [ ]:
# Make the program use only one GPU
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#set_session(tf.Session(config=config))


with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value


output_file_name='bitcoin2015to2017_close_CNN_2_relu'

step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]

epochs = 100

#split training validation
training_size = int(train_ratio* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:]
#build model

# 2 layers
model = Sequential()


model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
model.add(Dropout(0.5))
model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))

'''
# 3 Layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=8))
# 4 layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=2))
'''

model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels,verbose=1, batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs, callbacks=[CSVLogger(output_file_name+'.csv', append=True),ModelCheckpoint('weights/'+output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5', monitor='val_loss', verbose=1,mode='min')])

# Pick latest model weights
import glob
import os

list_of_files = glob.glob('weights/*') # * means all if need specific format then *.csv
latest_weights = max(list_of_files, key=os.path.getctime)
print(latest_weights)

In [ ]:
import glob
import os

list_of_files = glob.glob('weights/*') # * means all if need specific format then *.csv
latest_weights = max(list_of_files, key=os.path.getctime)
print(latest_weights)

In [ ]:
from keras import applications
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.callbacks import CSVLogger
import tensorflow as tf
#from scipy.ndimage import imread
#import scipy.imageio.imread
#from scipy.misc.pilutil import imread
import numpy as np
import random
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU
from keras import backend as K
import keras
from keras.callbacks import CSVLogger, ModelCheckpoint
#from keras.backend.tensorflow_backend import set_session
from keras import optimizers
import h5py
from sklearn.preprocessing import MinMaxScaler
import os
import pandas as pd
# import matplotlib

import matplotlib.pyplot as plt

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value
    input_times = hf['input_times'].value
    output_times = hf['output_times'].value
    original_inputs = hf['original_inputs'].value
    original_outputs = hf['original_outputs'].value
    original_datas = hf['original_datas'].value
    

scaler=MinMaxScaler()
#split training validation
training_size = int(train_ratio* datas.shape[0])
training_datas = datas[:training_size,:,:]
training_labels = labels[:training_size,:,:]
validation_datas = datas[training_size:,:,:]
validation_labels = labels[training_size:,:,:]
validation_original_outputs = original_outputs[training_size:,:,:]
validation_original_inputs = original_inputs[training_size:,:,:]
validation_input_times = input_times[training_size:,:,:]
validation_output_times = output_times[training_size:,:,:]

ground_true = np.append(validation_original_inputs,validation_original_outputs, axis=1)
ground_true_times = np.append(validation_input_times,validation_output_times, axis=1)
step_size = datas.shape[1]
batch_size= 15
nb_features = datas.shape[2]

model = Sequential()

# 2 layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
# model.add(LeakyReLU())
model.add(Dropout(0.25))
model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))
model.load_weights('{0}'.format(latest_weights))
model.compile(loss='mse', optimizer='adam')

In [ ]:
predicted = model.predict(validation_datas)
predicted_inverted = []

for i in range(original_datas.shape[1]):
    scaler.fit(original_datas[:,i].reshape(-1,1))
    predicted_inverted.append(scaler.inverse_transform(predicted[:,:,i]))
print (np.array(predicted_inverted).shape)
#get only the close data
ground_true = ground_true[:,:,0].reshape(-1)
ground_true_times = ground_true_times.reshape(-1)
ground_true_times = pd.to_datetime(ground_true_times, unit='s')
# since we are appending in the first dimension
predicted_inverted = np.array(predicted_inverted)[0,:,:].reshape(-1)
print (np.array(predicted_inverted).shape)
validation_output_times = pd.to_datetime(validation_output_times.reshape(-1), unit='s')

In [ ]:
def change_to_local(df, col_from = 'Timestamp', col_to = 'times',local_timezone = 'Singapore'):
    '''convert a datetime time unaware field to local
    '''
    from datetime import timezone, datetime

    # 3rd party modules
    import pandas as pd
    import pytz
    
    df[col_to] = pd.to_datetime(df[col_from], unit='s')
    df[col_to] = df[col_to].dt.tz_localize(timezone.utc)

    my_timezone = pytz.timezone(local_timezone)
    df[col_to] = df[col_to].dt.tz_convert(my_timezone)
    return df    

In [ ]:
ground_true_df = pd.DataFrame()
ground_true_df['times'] = ground_true_times
ground_true_df['value'] = ground_true

prediction_df = pd.DataFrame()
prediction_df['times'] = validation_output_times
prediction_df['value'] = predicted_inverted


# bring to local time
ground_true_df = change_to_local(ground_true_df, 'times','times', 'Singapore')
prediction_df= change_to_local(prediction_df, 'times','times', 'Singapore')

# build singular dataset
all_df = ground_true_df.merge(prediction_df, on = 'times', how = 'outer')
all_df.columns = ['times', 'actual', 'predicted']

#prediction_df = prediction_df.loc[(prediction_df["times"].dt.year == 2017 )&(prediction_df["times"].dt.month > 7 ),: ]
#ground_true_df = ground_true_df.loc[(ground_true_df["times"].dt.year == 2017 )&(ground_true_df["times"].dt.month > 7 ),:]

In [ ]:
# check original loaded data
container = change_to_local(container)
container.tail()

In [ ]:
# aggregate to date
gt = ground_true_df.groupby(pd.to_datetime(ground_true_df.times).dt.date)[['value']].mean().reset_index()
gt.columns = ['times', 'actual']
pt = prediction_df.groupby(pd.to_datetime(prediction_df.times).dt.date)[['value']].mean().reset_index()
pt.columns = ['times', 'predicted']
allt = gt.merge(pt, on = 'times', how = 'outer')

allt.set_index('times').plot(figsize=(20,5), grid=True, label = True)

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(ground_true_df.times,ground_true_df.value, label = 'Actual')
plt.plot(prediction_df.times,prediction_df.value,'ro--', label='Predicted')
plt.legend(loc='upper left')
plt.show()